In [1]:
from url import *

- We have the updated ingredients list with new quantities
- Need to map the quantities to the instructions
- Go through the instructions and look for ingredient measurement + descriptor + name

In [4]:
# url = "https://www.allrecipes.com/mediterranean-baked-cod-with-lemon-recipe-8576313"
url = "https://www.allrecipes.com/one-pot-chicken-pomodoro-recipe-8730087"
# url = 'https://www.allrecipes.com/shrimp-biryani-recipe-8679342'
# fetch_page_from_url(url)

### Test extract methods per step below ###

ingredients, instructions = fetch_page_from_url(url)

print(instructions)
print('\n')

transform_recipe(ingredients, instructions, to_vegetarian, scale=2.0)

# vegan_transform = transform_recipe(ingredients, instructions, to_vegetarian)
# print(vegan_transform['ingredients'])
# print(vegan_transform['instructions'])
# print('\n')

# healthy_transform = transform_recipe(ingredients, instructions, to_healthy)
# print(healthy_transform['ingredients'])
# print(healthy_transform['instructions'])
# print('\n')

# mexican_transform = transform_recipe(ingredients, instructions, mexican_style['ingredients'], mexican_style['techniques'])
# print(mexican_transform['ingredients'])
# print(mexican_transform['instructions'])
# print('\n')

# italian_transform = transform_recipe(ingredients, instructions, italian_style['ingredients'], italian_style['techniques'])
# print(italian_transform['ingredients'])
# print(italian_transform['instructions'])
# print('\n')

['Gather all ingredients', 'Heat a deep cast iron skillet over medium heat.', 'Season chicken breasts with salt, garlic, and Italian seasoning.', 'Pour 2 teaspoons oil into the skillet', 'Once hot, add chicken', 'Cook until golden, turning once, about 4 minutes per side', 'Move chicken to one side of the skillet.', 'Pour remaining oil into the skillet', 'Add minced shallot; cook until softened, about 3 minutes', 'Add the garlic, stir until fragrant, about 1 minute.', 'Add the crushed tomatoes, salt, sugar, red pepper flakes, and basil', 'Pour in chicken broth and wine', 'Stir to combine.', 'Add pasta to the skillet', 'Nestle chicken pieces on top of pasta', 'Reduce heat to medium-low', 'Cover, and cook until pasta is tender with a bite, about 8 minutes', 'Taste and adjust seasoning; serve immediately.']




{'ingredients': [{'quantity': '2',
   'unit': 'pound',
   'name': 'tofu',
   'preparation': 'thinly sliced'},
  {'quantity': '1.5', 'unit': 'teaspoon', 'name': ' salt'},
  {'quantity': '2', 'unit': 'teaspoon', 'name': ' garlic granules'},
  {'quantity': '2', 'unit': 'teaspoon', 'name': ' Italian seasoning'},
  {'quantity': '4',
   'unit': 'tablespoons',
   'name': ' , olive oil',
   'preparation': 'divided'},
  {'quantity': '2', 'name': ' , shallot', 'preparation': 'minced'},
  {'quantity': '8',
   'unit': 'cloves',
   'name': ' , garlic',
   'preparation': 'minced'},
  {'quantity': '2',
   'unit': '(28 ounce) can',
   'name': ' can tomatoes',
   'preparation': 'crushed'},
  {'quantity': '0.75', 'unit': 'teaspoon', 'name': ' salt'},
  {'quantity': '2', 'unit': 'teaspoon', 'name': ' white sugar'},
  {'quantity': '0.75',
   'unit': 'teaspoon',
   'name': ' red pepper flakes',
   'preparation': 'crushed'},
  {'quantity': '2',
   'unit': 'tablespoon',
   'name': ' fresh basil',
   'prepara

In [ ]:



### Handles fractions in ingredients list

def parse_fraction(quantity_str):
    parts = quantity_str.split()
    if len(parts) == 1:
        if '/' in parts[0]:
            numerator, denominator = parts[0].split('/')
            return Fraction(int(numerator), int(denominator))
        else:
            return Fraction(str(float(parts[0])))
    else:
        whole = int(parts[0])
        frac_part = parts[1]
        numerator, denominator = frac_part.split('/')
        return Fraction(whole * int(denominator) + int(numerator), int(denominator))

def format_fraction(frac):
    if frac.denominator == 1:
        return str(frac.numerator)
    else:
        whole = frac.numerator // frac.denominator
        remainder = frac.numerator % frac.denominator
        if whole == 0:
            return f"{remainder}/{frac.denominator}"
        else:
            return f"{whole} {remainder}/{frac.denominator}"

In [3]:
def substr(word, list):
    for i in list:
        if word in i or i in word:
            return True
    return False

def scale_instructions(ingredients, instructions, factor):
    if not factor:
        return instructions

    measurements = set()
    ingredient_names = set()
    for ing in ingredients:
        if "unit" in ing and ing["unit"]:
            measurements.add(ing["unit"].lower())

        if "name" in ing and ing["name"]:
            # first_word = ing["name"].split()[0].lower()
            ingredient_names.add(ing["name"])

    scaled_instructions = []

    for line in instructions:
        words = line.split()
        i = 0
        while i < len(words):
            word = words[i]
            
            # Handles mixed fractions (i.e. 1 1/2)
            if i + 1 < len(words):
                try:
                    whole_num = float(word)
                    next_token = words[i+1]
                    if '/' in next_token:
                        combined = f"{word} {next_token}"
                        del words[i+1]
                        word = combined
                except ValueError:
                    pass
            
            # look for fractions
            parsed_value = None
            try:
                parsed_value = parse_fraction(word)
            except:
                pass
            
            # check if next word is found in our ingredients list (i.e. teaspoon)
            if parsed_value is not None and i + 1 < len(words):
                next_word = words[i+1].lower()
                if next_word != "to" and next_word != "F" and (substr(next_word, measurements) or substr(next_word, ingredient_names)):
                    new_quantity = parsed_value * factor
                    if new_quantity.is_integer():
                        words[i] = str(int(new_quantity))
                    else:
                        words[i] = str(new_quantity)
             

            i += 1

        scaled_instructions.append(" ".join(words))

    return scaled_instructions

In [5]:
test = {"a":1}
test2 = {"b":2}
print("True") if test or test2 else print("False")

True


In [4]:
instructions

['Gather all ingredients.',
 'Place rice in a large bowl, and cover with cold running water',
 'Agitate with hands to release excess starch, and pour off water',
 'Repeat process until water is mostly clear, 4 to 5 times.',
 'Heat 2 tablespoons of the ghee in a medium saucepan over medium-high heat',
 'Add 1 tablespoon garam masala, 2 teaspoons salt, 1 teaspoon each of grated garlic and ginger, and 1/2 teaspoon coriander',
 'Cook, stirring constantly, until fragrant, about 1 minute',
 'Add rice, and cook, stirring constantly, until rice is well coated in ghee and spices, 1 to 2 minutes.',
 'Add 2 1/4 cups water, and stir',
 'Bring to a boil over high heat',
 'Stir, and cover with a tight-fitting lid',
 'Reduce heat to medium-low',
 'Simmer, undisturbed, until rice is just tender, about 12 minutes',
 'Remove from heat, and remove lid',
 'Fluff with a fork, and set aside.',
 'Meanwhile, whisk together yogurt, cayenne, 1/2 teaspoon garam masala, and 1/2 teaspoon salt',
 'Add shrimp, and t

In [5]:
scale_instructions(ingredients, instructions, 2.0)

['Gather all ingredients.',
 'Place rice in a large bowl, and cover with cold running water',
 'Agitate with hands to release excess starch, and pour off water',
 'Repeat process until water is mostly clear, 4 to 5 times.',
 'Heat 4 tablespoons of the ghee in a medium saucepan over medium-high heat',
 'Add 2 tablespoon garam masala, 4 teaspoons salt, 2 teaspoon each of grated garlic and ginger, and 1 teaspoon coriander',
 'Cook, stirring constantly, until fragrant, about 1 minute',
 'Add rice, and cook, stirring constantly, until rice is well coated in ghee and spices, 1 to 2 minutes.',
 'Add 4.5 cups water, and stir',
 'Bring to a boil over high heat',
 'Stir, and cover with a tight-fitting lid',
 'Reduce heat to medium-low',
 'Simmer, undisturbed, until rice is just tender, about 12 minutes',
 'Remove from heat, and remove lid',
 'Fluff with a fork, and set aside.',
 'Meanwhile, whisk together yogurt, cayenne, 1 teaspoon garam masala, and 1 teaspoon salt',
 'Add shrimp, and toss to f

In [46]:
line = 'Season shredded chicken breasts with Tajín, garlic, and Italian seasoning.'
line_lower = line.lower()
for old_ing, new_ing in mexican_style['ingredients'].items():
    print(old_ing, old_ing in line_lower)
    if old_ing in line_lower:
        line = line_lower.replace(old_ing, ' or '.join(new_ing))

line

butter False
ghee False
olive oil False
sesame oil False
vegetable oil False
bread False
rice False
noodles False
couscous False
polenta False
flatbread False
dumplings False
wraps False
potatoes False
tofu False
paneer False
ground meat False
chicken True
fish False
cheddar False
parmesan False
feta False
mozzarella False
cream False
sour cream False
yogurt False
lemon juice False
vinegar False
balsamic vinegar False
soy sauce False
fish sauce False
bell peppers False
jalapeños False
okra False
bok choy False
scallions False
spinach False
cilantro False
mint False
pickled vegetables False
chutney False
tortilla chips False
croutons False
herbes de provence False
italian seasoning True
basil False
parsley False
dill False
sage False
oregano False
garam masala False
cayenne pepper False
paprika False
chili powder False
red pepper flakes False
ground chili False
salt False
Worcestershire sauce False
lemon zest False
bay leaves False


'season shredded chicken breasts with tajín, garlic, and cumin.'

In [3]:
alternatives = find_ingredients(ingredients, to_vegetarian)
alternatives

{'cod filets': ['tofu', 'seaweed-marinated tofu']}

In [4]:
replace_ingredients(ingredients, alternatives)

[{'quantity': '3',
  'unit': 'tablespoons',
  'name': ' butter',
  'descriptor': 'unsalted',
  'preparation': 'softened'},
 {'quantity': '1',
  'unit': 'tablespoon',
  'name': ' garlic',
  'preparation': 'finely minced',
  'descriptor': 'fresh'},
 {'quantity': '1',
  'unit': 'tablespoon',
  'name': ' parsley',
  'preparation': 'minced',
  'descriptor': 'fresh'},
 {'quantity': '2',
  'unit': 'teaspoons',
  'name': ' oregano',
  'preparation': 'minced',
  'descriptor': 'fresh'},
 {'quantity': '1',
  'unit': 'teaspoon',
  'name': ' thyme',
  'alternative': 'rosemary',
  'preparation': 'minced',
  'descriptor': 'fresh'},
 {'quantity': '1/2',
  'unit': 'teaspoon',
  'name': ' salt',
  'alternative': 'to taste',
  'preparation': 'finely ground',
  'descriptor': 'pink'},
 {'quantity': '1/4',
  'unit': 'teaspoon',
  'name': ' pepper',
  'preparation': 'freshly ground',
  'descriptor': 'black'},
 {'quantity': '1/2', 'unit': 'teaspoon', 'name': ' paprika'},
 {'quantity': '4', 'unit': '(6 ounce)'

In [5]:
ingredients

[{'quantity': '3',
  'unit': 'tablespoons',
  'name': ' butter',
  'descriptor': 'unsalted',
  'preparation': 'softened'},
 {'quantity': '1',
  'unit': 'tablespoon',
  'name': ' garlic',
  'preparation': 'finely minced',
  'descriptor': 'fresh'},
 {'quantity': '1',
  'unit': 'tablespoon',
  'name': ' parsley',
  'preparation': 'minced',
  'descriptor': 'fresh'},
 {'quantity': '2',
  'unit': 'teaspoons',
  'name': ' oregano',
  'preparation': 'minced',
  'descriptor': 'fresh'},
 {'quantity': '1',
  'unit': 'teaspoon',
  'name': ' thyme',
  'alternative': 'rosemary',
  'preparation': 'minced',
  'descriptor': 'fresh'},
 {'quantity': '1/2',
  'unit': 'teaspoon',
  'name': ' salt',
  'alternative': 'to taste',
  'preparation': 'finely ground',
  'descriptor': 'pink'},
 {'quantity': '1/4',
  'unit': 'teaspoon',
  'name': ' pepper',
  'preparation': 'freshly ground',
  'descriptor': 'black'},
 {'quantity': '1/2', 'unit': 'teaspoon', 'name': ' paprika'},
 {'quantity': '4', 'unit': '(6 ounce)'

In [9]:
line = "Pat cod filets dry"
for old_ing, new_ing in to_vegetarian.items():
    if old_ing in line.lower():
        line = line.replace(old_ing, ' or '.join(new_ing))
line

'Pat tofu or seaweed-marinated tofu filets dry'

In [8]:
from pyfood.utils import Shelf
shelf = Shelf()

In [9]:
for i in ingredients:
    print(i)
    print(shelf.process_ingredients([i['name'] + i['descriptor'] if 'discriptor' in i.keys() else i['name']]))

{'quantity': '3', 'unit': 'tablespoons', 'name': ' butter', 'descriptor': 'unsalted', 'preparation': 'softened'}
{'ingredients': [{'foodname': 'butter', 'quantity': 100.0, 'unit': 'g', 'taxon': '201'}], 'HS': [], 'ingredients_by_taxon': [['butter']], 'labels': {'vegan': False, 'vege': True, 'seasonality': {'city': 'EU', 'score': 50, 'best_now': True}}, 'revisited': []}
{'quantity': '1', 'unit': 'tablespoon', 'name': ' garlic', 'preparation': 'finely minced', 'descriptor': 'fresh'}
{'ingredients': [{'foodname': 'garlic', 'quantity': 100.0, 'unit': 'g', 'taxon': '002'}], 'HS': [], 'ingredients_by_taxon': [['garlic']], 'labels': {'vegan': True, 'vege': True, 'seasonality': {'city': 'EU', 'score': 100, 'best_now': True}}, 'revisited': []}
{'quantity': '1', 'unit': 'tablespoon', 'name': ' parsley', 'preparation': 'minced', 'descriptor': 'fresh'}
{'ingredients': [{'foodname': 'parsley', 'quantity': 100.0, 'unit': 'g', 'taxon': '008'}], 'HS': [], 'ingredients_by_taxon': [['parsley']], 'labels

NotFittedError: This TfidfTransformer instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [10]:
shelf.process_ingredients(['cod filet'])

NotFittedError: This TfidfTransformer instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.